In [2]:
from transformers import pipeline

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Cargar pipeline de análisis de sentimiento
sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
import pandas as pd

# Cargamos el dataset
df = pd.read_csv('raw/fichajes_news.csv', sep=';')
df.head()

,title,date,content
0,El Real Madrid lo tiene claro: negativa al AC ...,01/08,El Real Madrid ha rechazado un acercamiento po...
1,La Atalanta estudia moverse por una pieza de l...,01/08,Gian Piero Gasperini confía en potenciar toda...
2,La Juventus estudia un fichaje de 20 M€,01/08,Decidida a mejorar las prestaciones de su ataq...
3,Osasuna confirma la venta de David García,01/08,Se venía apuntando a esta posibilidad desde h...
4,El Atlético busca la fórmula para fichar a Al...,01/08,Aunque Alexander Sörloth ya ha dado el sí qui...


In [5]:
df['whole'] = df['title'] + ' ' + df['content']
df['whole'] = df['whole'].astype(str)
df.head()

,title,date,content,whole
0,El Real Madrid lo tiene claro: negativa al AC ...,01/08,El Real Madrid ha rechazado un acercamiento po...,El Real Madrid lo tiene claro: negativa al AC ...
1,La Atalanta estudia moverse por una pieza de l...,01/08,Gian Piero Gasperini confía en potenciar toda...,La Atalanta estudia moverse por una pieza de l...
2,La Juventus estudia un fichaje de 20 M€,01/08,Decidida a mejorar las prestaciones de su ataq...,La Juventus estudia un fichaje de 20 M€ Decidi...
3,Osasuna confirma la venta de David García,01/08,Se venía apuntando a esta posibilidad desde h...,Osasuna confirma la venta de David García Se ...
4,El Atlético busca la fórmula para fichar a Al...,01/08,Aunque Alexander Sörloth ya ha dado el sí qui...,El Atlético busca la fórmula para fichar a Al...


In [6]:
# Dividimos en chunks de 256 palabras
def chunk_text(text, chunk_size=256):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    return chunks

df['chunks'] = df['whole'].apply(chunk_text)
df.head()

,title,date,content,whole,chunks
0,El Real Madrid lo tiene claro: negativa al AC ...,01/08,El Real Madrid ha rechazado un acercamiento po...,El Real Madrid lo tiene claro: negativa al AC ...,[El Real Madrid lo tiene claro: negativa al AC...
1,La Atalanta estudia moverse por una pieza de l...,01/08,Gian Piero Gasperini confía en potenciar toda...,La Atalanta estudia moverse por una pieza de l...,[La Atalanta estudia moverse por una pieza de ...
2,La Juventus estudia un fichaje de 20 M€,01/08,Decidida a mejorar las prestaciones de su ataq...,La Juventus estudia un fichaje de 20 M€ Decidi...,[La Juventus estudia un fichaje de 20 M€ Decid...
3,Osasuna confirma la venta de David García,01/08,Se venía apuntando a esta posibilidad desde h...,Osasuna confirma la venta de David García Se ...,[Osasuna confirma la venta de David García Se...
4,El Atlético busca la fórmula para fichar a Al...,01/08,Aunque Alexander Sörloth ya ha dado el sí qui...,El Atlético busca la fórmula para fichar a Al...,[ El Atlético busca la fórmula para fichar a A...


In [12]:
# Analizamos el sentimiento de cada chunk
df['sentiment'] = df['chunks'].apply(sentiment_analysis)

In [15]:
def score(sentiments):
    return sum([s['score'] if s['label'] == 'POSITIVE' else -s['score'] for s in sentiments]) / len(sentiments)

df['score'] = df['sentiment'].apply(score)

-0.7333669993612502

3519 columna final temporada o 809

In [7]:
# Analizamos el sentimiento de cada chunk (comprehension)
df['sentiment'] = df['chunks'].apply(lambda x: [sentiment_analysis(chunk) for chunk in x])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\david\AppData\Local\Temp\ipykernel_34452\1329239830.py", line 9, in <module>
    df['sentiments'] = df['chunks'].apply(analyze_chunks)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\series.py", line 4760, in apply
    ).apply()
      ^^^^^^^
  File "C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\apply.py", line 1207, in apply
    return self.apply_standard()
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\davi